In [1]:
#startup template for Jupyter Notebook
# activate tab completion so I won't shy away from descriptive vars
%config IPCompleter.use_jedi = False
%config IPCompleter.greedy = True
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
import collections
import warnings
from datetime import date, time, datetime, timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.dates as mdates
from IPython.display import display, HTML, Markdown, Video, Javascript, Image
# Allow for multiple outputs from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  # 'last' for last output other options: (last_expr, last_expr_or_assign, 'None')   
warnings.filterwarnings('ignore')

To start with i generate a list which maps the chain IDs used by Li.Fi. There was no particular need to use Polygon's event logs table, but it gives me what I need. We want human-readable labels

### Chain Ids table

The below SQL script maps the chain ids used by Li.Fi to the human readable chain label. The choice of the polygon event logs table was arbitrary.

```sql
with chain_ids as (
select distinct decoded_log:bridgeData:destinationChainId as dst_chain,
                case
                  when dst_chain = 1 then 'ethereum'
                  when dst_chain = 8453 then 'base'
                  when dst_chain = 137 then 'polygon'
                  when dst_chain = 42161 then 'arbitrum'
                  when dst_chain = 10 then 'optimism'
                  when dst_chain = 56 then 'bnb'
                  when dst_chain = 43114 then 'avalanche'
                  when dst_chain = 1101 then 'pol zkevm'
                  when dst_chain = 59144 then 'linea'
                  when dst_chain = 100 then 'gnosis'
                  when dst_chain = 250 then 'fantom'
                  when dst_chain = 5000 then 'mantle'
                  when dst_chain = 1313161554 then 'aurora'
                  when dst_chain = 7777777 then 'zora'
                  when dst_chain = 324 then 'zksync'
                  when dst_chain = 1151111081099710 then 'solana'
                  when dst_chain = 534352 then 'scroll'
                  when dst_chain = 34443 then 'mode'
                  else 'Unknown'
                end as chain

from polygon.core.ez_decoded_event_logs 
where  event_name = 'LiFiTransferStarted'
 
)
select * from chain_ids
       

In [5]:
chainIDs_df = pd.read_csv('data/chainIDs.csv')
chainIDs_df

,DST_CHAIN,CHAIN
0,42161,arbitrum
1,56,bnb
2,100,gnosis
3,250,fantom
4,324,zksync
5,10,optimism
6,59144,linea
7,5000,mantle
8,1088,Unknown
9,81457,Unknown


We won't be using that because we are just going to incorporate it into our SQL query.

### Unioning all the schemas

In [22]:
# chains = ['arbitrum', 'bsc', 'optimism', 'base', 'avalanche', 'ethereum','polygon']
# chains.sort()
# sql = ""
# for chain in chains:
#     query[chain] = 
chain = 'ethereum'
sql = f"""
{chain}_raw_bridge_data as (
          select decoded_log:bridgeData:destinationChainId as dst_chain_id,
                 decoded_log:bridgeData:bridge as bridge,
                 decoded_log:bridgeData:integrator as integrator,
                 tx_hash
          from {chain}.core.ez_decoded_event_logs
          where event_name =  'LiFiTransferStarted'
            and block_timestamp > current_timestamp - interval '30 days'
),
{chain}_bridge_data as (
  select '{chain}' as source_chain,
         c.dst_chain,
         r.bridge,
         r.integrator,
         r.tx_hash
  from chain_ids c
        left join 
        {chain}_raw_bridge_data r 
        using
        (dst_chain_id)

"""
print(sql.strip())


ethereum_raw_bridge_data as (
          select decoded_log:bridgeData:destinationChainId as dst_chain_id,
                 decoded_log:bridgeData:bridge as bridge,
                 decoded_log:bridgeData:integrator as integrator,
                 tx_hash
          from ethereum.core.ez_decoded_event_logs
          where event_name =  'LiFiTransferStarted'
            and block_timestamp > current_timestamp - interval '30 days'
),
ethereum_bridge_data as (
  select 'ethereum' as source_chain,
         c.dst_chain,
         r.bridge,
         r.integrator,
         r.tx_hash
  from chain_ids c
        left join 
        ethereum_raw_bridge_data r 
        using
        (dst_chain_id)
)


I created the above script to facilitate a rather repetitive process, querying the tables from each chain's schema

### Transfer tables

In [24]:
# chains = ['arbitrum', 'bsc', 'optimism', 'base', 'avalanche', 'ethereum','polygon']
sql = ""
for chain in chains:
    query = f"""
    {chain}_transfers as (
    select block_timestamp, 
    	  from_address ,
    	  amount_usd, 
    	  'native' as source_token, 
    	  tx_hash 
     from {chain}.core.ez_native_transfers
     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
       and block_timestamp > current_timestamp - interval '30 days'
       and amount_usd >= 20
    
    union all
    
    select block_timestamp, 
    	  from_address ,
    	  amount_usd, 
    	  symbol as source_token, 
    	  tx_hash 
    from  {chain}.core.ez_token_transfers
    where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
      and block_timestamp > current_timestamp - interval '30 days'
      and amount_usd >= 20
    ),
    """
    sql = sql + '\n' + query
print(sql)



    arbitrum_transfers as (
    select block_timestamp, 
    	  from_address ,
    	  amount_usd, 
    	  'native' as source_token, 
    	  tx_hash 
     from arbitrum.core.ez_native_transfers
     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
       and block_timestamp > current_timestamp - interval '30 days'
       and amount_usd >= 20
    
    union all
    
    select block_timestamp, 
    	  from_address ,
    	  amount_usd, 
    	  symbol as source_token, 
    	  tx_hash 
    from  arbitrum.core.ez_token_transfers
    where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
      and block_timestamp > current_timestamp - interval '30 days'
      and amount_usd >= 20
    ),
    

    avalanche_transfers as (
    select block_timestamp, 
    	  from_address ,
    	  amount_usd, 
    	  'native' as source_token, 
    	  tx_hash 
     from avalanche.core.ez_native_transfers
     where to_address = '0x1231deb6f5749ef6ce6943a275a1d3e7486f4eae'
       and bloc

Almost there! We now union all these tables together

### Data collection

Columns needed: 
  - timestamp
  - receiver
  - source_chain
  - destination_chain   
  - amount usd (need to merge with price table
  - token_address
  - token symbol

~~furrow~~